In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
# Read in data
accounts = pd.read_csv("../Resources/accounts.csv")
ast = pd.read_csv("../Resources/account_state_transitions.csv")
groups = pd.read_csv("../Resources/groups.csv")

## 4) Devise a method of generating a table of all Accounts with their “nominal” expected payments as of any arbitrary timestamp in the past. In other words, we’re looking for the amount the distributor would have collected as of the test date assuming the client had paid the minimum payment instantaneously upon going to a “disabled” state until reaching the unlock price.

## Merge Dataframes

In [3]:
# Merge 'ast' and 'accounts' dataframes
merged_df = pd.merge(left=ast, right=accounts, how='outer', left_on='account_id', right_on='id')

# Limit columns
merged_df = merged_df[['started_when','account_id','to_state','group_id']]
merged_df.head()

In [4]:
# Merge with 'groups' dataframe
merged_df = pd.merge(left=merged_df, right=groups, how='outer', left_on='group_id', right_on='id')

# Limit columns
merged_df = merged_df.drop(columns=['id','name','price_clock_hour'])
merged_df.head()

,started_when,account_id,to_state,group_id,price_upfront,price_unlock,minimum_payment
0,2019-11-06 05:49:39.571392,2,ENABLED,1,950,8100,71.5
1,2019-11-25 22:20:59.150339,2,DISABLED,1,950,8100,71.5
2,2019-11-26 10:22:36.571392,2,ENABLED,1,950,8100,71.5
3,2019-11-29 11:03:32.150339,2,DISABLED,1,950,8100,71.5
4,2019-12-03 10:08:03.571392,2,ENABLED,1,950,8100,71.5


## Limit by Indicated Timestamp

In [5]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [6]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

# Adjust timestamp to end of day
timestamp_dt = timestamp_dt + timedelta(hours=23,minutes=59,seconds=59)

In [7]:
# Change data type from string to datetime object
merged_df['started_when'] = pd.to_datetime(merged_df['started_when'])

In [8]:
# Create limited dataframe based on indicated timestamp and to_state
merged_limited_df = merged_df.loc[(merged_df['started_when'] <= timestamp_dt) & (merged_df['to_state']=='DISABLED')]

In [9]:
merged_limited_df.head()

,started_when,account_id,to_state,group_id,price_upfront,price_unlock,minimum_payment
1,2019-11-25 22:20:59.150339,2,DISABLED,1,950,8100,71.5
3,2019-11-29 11:03:32.150339,2,DISABLED,1,950,8100,71.5
5,2019-12-06 06:44:56.624024,2,DISABLED,1,950,8100,71.5
7,2019-12-12 11:56:56.518760,2,DISABLED,1,950,8100,71.5
9,2019-12-18 11:33:11.202971,2,DISABLED,1,950,8100,71.5


In [ ]:
# Group by account and count the occurrences of "DISABLED" state 
disabled_grouped_df = disabled_limited_df.groupby('account_id',as_index=False).agg({
    'to_state':'count',
    'price_upfront':'max',
    'price_unlock':'max',
    'minimum_payment':'max',})

# Rename column
disabled_grouped_df = disabled_grouped_df.rename(columns={'to_state':'count_disabled'})
disabled_grouped_df.head()

In [ ]:
# Calculate nominal expected payment
expected_payment = (disabled_grouped_df['count_disabled']*disabled_grouped_df['minimum_payment']) + disabled_grouped_df['price_upfront']

In [ ]:
# Make new dataframe with account_id and expected_payment
df = pd.DataFrame({'account_id':disabled_grouped_df['account_id'],
                  f'expected_payment_as_of_{timestamp_str}':expected_payment})

# Preview dataframe
print(f"NOMINAL EXPECTED PAYMENTS BY ACCOUNT AS OF: {timestamp_str[:10]}")
df.head()

In [ ]:
# Export to csv
df.to_csv(f"../Output/nominal_expected_payments_as_of_{timestamp_str[:10]}.csv", index=False)